In [92]:
import yfinance as yf
import pandas as pd
import pandas_ta as ta

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler



In [93]:
# Step 2: Get Historical Data
gspc_hist = pd.DataFrame(hist)

In [47]:
# calculating technical indicators 
hist["RSI"] = ta.rsi(hist["Close"], length=14)
hist["MACD"] = ta.macd(hist["Close"]).iloc[:, 0]
hist["SMA_20"] = hist["Close"].rolling(20).mean()

In [ ]:

clean_hist = hist.dropna()
clean_hist.head(5)

,Open,High,Low,Close,Volume,Dividends,Stock Splits,RSI,MACD,SMA_20
Date,,,,,,,,,,
2020-06-16 00:00:00-04:00,3131.000000,3153.449951,3076.060059,3124.739990,5845810000,0.0,0.0,57.849813,66.719423,3066.530005
2020-06-17 00:00:00-04:00,3136.129883,3141.159912,3108.030029,3113.489990,4560450000,0.0,0.0,56.550585,63.602842,3076.057507
2020-06-18 00:00:00-04:00,3101.639893,3120.000000,3093.510010,3115.340088,4448690000,0.0,0.0,56.722720,60.583844,3083.244006
2020-06-19 00:00:00-04:00,3140.290039,3155.530029,3083.110107,3097.739990,8346650000,0.0,0.0,54.510287,56.124121,3090.705505
2020-06-22 00:00:00-04:00,3094.419922,3120.919922,3079.389893,3117.860107,4686690000,0.0,0.0,56.594573,53.595469,3098.826013


In [75]:
# Extract and display unique years from the index
years = clean_hist.index.year.unique()
print(sorted(years))


[2020, 2021, 2022, 2023, 2024, 2025]


In [50]:
# === Step 3: Create Target Variable ===
# Label as 1 (Buy) if price rises >2% over next 5 days, else 0 (Hold), -1 (Sell)
future_days = 5
threshold = 0.02

clean_hist['future_return'] = clean_hist['Close'].shift(-future_days) / clean_hist['Close'] - 1

def label_trade(x):
    if x > threshold:
        return 1   # Buy
    elif x < -threshold:
        return -1  # Sell
    else:
        return 0   # Hold

clean_hist['target'] = clean_hist['future_return'].apply(label_trade)

/var/folders/qx/zjy9jsyd2sg69nyf65kyxhyr0000gn/T/ipykernel_7406/1811243750.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_hist['future_return'] = clean_hist['Close'].shift(-future_days) / clean_hist['Close'] - 1
/var/folders/qx/zjy9jsyd2sg69nyf65kyxhyr0000gn/T/ipykernel_7406/1811243750.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_hist['target'] = clean_hist['future_return'].apply(label_trade)


In [51]:
# === Step 4: Drop NaNs ===
clean_hist = clean_hist.dropna(subset=['RSI', 'MACD', 'SMA_20', 'target'])

In [54]:
# === Step 5: Define Features (X) and Labels (y) ===
X = clean_hist[['RSI', 'MACD', 'SMA_20']]
y = (clean_hist['Close'].shift(-5) > clean_hist['Close']).astype(int)  # 1 if price increases in next 5 days, else 0
X.dropna(inplace=True)
y = y.loc[X.index]  # Align y with X after dropping


/var/folders/qx/zjy9jsyd2sg69nyf65kyxhyr0000gn/T/ipykernel_7406/2071627570.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.dropna(inplace=True)


In [55]:
# === Step 6: Train/Test Split ===
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [56]:
# === Step 7: Train Model ===
model = RandomForestClassifier()
model.fit(X_train, y_train)

RandomForestClassifier()

In [57]:
# === Step 8: Predict and Evaluate ===
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.63      0.50      0.56        96
           1       0.72      0.81      0.76       151

    accuracy                           0.69       247
   macro avg       0.68      0.66      0.66       247
weighted avg       0.69      0.69      0.68       247



In [58]:
# Drop unused columns
clean_hist = clean_hist.drop(columns=['Dividends', 'Stock Splits'])

In [60]:
# Step 1: Create future_return
hist['future_return'] = (hist['Close'].shift(-7) - hist['Close']) / hist['Close']

# Step 2: Create label column
hist['label'] = (hist['future_return'] > 0.05).astype(int)

# Step 3: Drop rows with NaNs in indicators and future return
clean_hist = hist.dropna(subset=['RSI', 'MACD', 'SMA_20', 'future_return'])



In [61]:
X = clean_hist[['RSI', 'MACD', 'SMA_20']]
y = clean_hist['label']


In [63]:
# Apply PCA After Feature Selection to reduce noise and dimensionality in your feature set:
#Step 1: Scale the Features

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


In [64]:
#Step 2: Apply PCA
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)


In [65]:
#Step 4: Check Explained Variance
print("Explained variance ratio:", pca.explained_variance_ratio_)
print("Total explained variance:", pca.explained_variance_ratio_.sum())


Explained variance ratio: [0.60327178 0.33257554]
Total explained variance: 0.9358473195151563


In [66]:
#Step 5: Continue With Train/Test Split and Model
X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.2, random_state=42)

model = RandomForestClassifier()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.97      1.00      0.99       238
           1       0.00      0.00      0.00         7

    accuracy                           0.97       245
   macro avg       0.49      0.50      0.49       245
weighted avg       0.94      0.97      0.96       245



/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [68]:
# Check Class Distribution
# If class 1 has very few samples, it's hard for the model to learn from them.
print(y.value_counts())


label
0    1186
1      39
Name: count, dtype: int64


In [69]:
# Use Class Weights
model = RandomForestClassifier(class_weight='balanced')


In [70]:
# Use Class Weights
from imblearn.over_sampling import SMOTE

smote = SMOTE()
X_resampled, y_resampled = smote.fit_resample(X_pca, y)


In [71]:
 # Use Stratified Split. 
 # Ensure train/test split keeps class proportions:
X_train, X_test, y_train, y_test = train_test_split(
    X_resampled, y_resampled, test_size=0.2, random_state=42, stratify=y_resampled
)


In [72]:
# Step 1: Define your features and labels
X = clean_hist[['RSI', 'MACD', 'SMA_20']]
y = clean_hist['label']

# Step 2: Apply PCA (optional but can reduce noise)
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X)

# Step 3: Handle class imbalance with SMOTE
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_pca, y)

# Step 4: Train/test split (stratified)
X_train, X_test, y_train, y_test = train_test_split(
    X_resampled, y_resampled, test_size=0.2, random_state=42, stratify=y_resampled
)

# Step 5: Train model
model = RandomForestClassifier()
model.fit(X_train, y_train)

# Step 6: Predict and evaluate
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.97      0.95      0.96       238
           1       0.95      0.97      0.96       237

    accuracy                           0.96       475
   macro avg       0.96      0.96      0.96       475
weighted avg       0.96      0.96      0.96       475



In [87]:
#use this for the tick and then use chatgpt


import yfinance as yf
import pandas as pd

# === Step 1: Download NVDA data ===
nvda_df = yf.download("NVDA", start="2020-01-01", end="2025-01-01")

# === Step 2: Reset index for merging or future use ===
nvda_df.reset_index(inplace=True)

# === Step 3: Preview ===
print(nvda_df.head())



# nvda = pd.DataFrame(hist)


[*********************100%***********************]  1 of 1 completed

Price        Date     Close      High       Low      Open     Volume
Ticker                 NVDA      NVDA      NVDA      NVDA       NVDA
0      2020-01-02  5.972162  5.972162  5.892753  5.943286  237536000
1      2020-01-03  5.876570  5.920383  5.827530  5.852424  205384000
2      2020-01-06  5.901214  5.906442  5.757082  5.783220  262636000
3      2020-01-07  5.972661  6.018464  5.884538  5.929595  314856000
4      2020-01-08  5.983862  6.025185  5.928350  5.968428  277108000


In [90]:

# hist["RSI"] = ta.rsi(hist["Close"], length=14)
# hist["MACD"] = ta.macd(hist["Close"]).iloc[:, 0]
# hist["SMA_20"] = hist["Close"].rolling(20).mean()

import pandas_ta as ta

# Assuming 'hist' is your DataFrame for NVDA
hist["RSI"] = ta.rsi(hist["Close"], length=14)

# MACD returns a DataFrame with multiple columns, we'll take the main MACD line
macd = ta.macd(hist["Close"])
hist["MACD"] = macd["MACD_12_26_9"]  # You can also get 'MACDs_12_26_9' and 'MACDh_12_26_9' if needed

# 20-day Simple Moving Average
hist["SMA_20"] = hist["Close"].rolling(window=20).mean()

# Drop NaNs from indicator calculations
hist.dropna(subset=["RSI", "MACD", "SMA_20"], inplace=True)


hist.head(10)  # Shows the first 10 rows


,Open,High,Low,Close,Volume,Dividends,Stock Splits,RSI,MACD,SMA_20,future_return,label
Date,,,,,,,,,,,,
2020-08-26 00:00:00-04:00,3449.969971,3481.070068,3444.149902,3478.729980,3780530000,0.0,0.0,80.006249,63.909742,3362.519019,-0.014882,0
2020-08-27 00:00:00-04:00,3485.139893,3501.379883,3468.350098,3484.550049,3955890000,0.0,0.0,80.561123,66.056544,3374.435522,-0.043825,0
2020-08-28 00:00:00-04:00,3494.689941,3509.229980,3484.320068,3508.010010,3868510000,0.0,0.0,82.651161,68.857181,3386.280017,-0.031086,0
2020-08-31 00:00:00-04:00,3509.729980,3514.770020,3493.250000,3500.310059,4348280000,0.0,0.0,79.625096,69.652473,3396.565015,-0.046030,0
2020-09-01 00:00:00-04:00,3507.439941,3528.030029,3494.600098,3526.649902,4101490000,0.0,0.0,82.046604,71.582989,3407.572009,-0.052651,0
2020-09-02 00:00:00-04:00,3543.760010,3588.110107,3535.229980,3580.840088,4295640000,0.0,0.0,85.788709,76.602605,3420.225513,-0.055099,0
2020-09-03 00:00:00-04:00,3564.739990,3564.850098,3427.409912,3455.060059,4911830000,0.0,0.0,56.402503,69.628654,3425.520520,-0.015589,0
2020-09-04 00:00:00-04:00,3453.600098,3479.149902,3349.629883,3426.959961,4438720000,0.0,0.0,52.108131,61.129640,3429.304517,-0.012101,0
2020-09-08 00:00:00-04:00,3371.879883,3379.969971,3329.270020,3331.840088,4694330000,0.0,0.0,40.787380,46.186324,3427.873022,0.007554,0
